In [ ]:
# Install necessary libraries (if not already installed)
%pip install transformers datasets peft evaluate scikit-learn tqdm

In [ ]:
%pip install ipywidgets
%pip install jupyterlab_widgets

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, accuracy_score, mean_squared_error
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch.optim import AdamW

In [ ]:
# Paths
DATASET_PATH = '/Users/rumenguin/Research/MERC/EmoReact/dataset.csv'
TRAIN_NAMES_PATH = '/Users/rumenguin/Research/MERC/EmoReact/Labels/Train_names.txt'
VAL_NAMES_PATH = '/Users/rumenguin/Research/MERC/EmoReact/Labels/Val_names.txt'
TEST_NAMES_PATH = '/Users/rumenguin/Research/MERC/EmoReact/Labels/Test_names.txt'
MODEL_NAME = 'distilbert-base-uncased'  # Good balance of performance and efficiency for Mac M1
SAVE_DIR = '/Users/rumenguin/Research/MERC/models/saved_model'

# Create save directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

# Set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Load and preprocess the dataset
def load_video_names(file_path):
    with open(file_path, 'r') as f:
        # Remove quotes if present and strip whitespace
        return [line.strip().replace("'", "").replace('"', '') for line in f.readlines()]

# Load video names for train, val, and test sets
train_videos = load_video_names(TRAIN_NAMES_PATH)
val_videos = load_video_names(VAL_NAMES_PATH)
test_videos = load_video_names(TEST_NAMES_PATH)

print(f"Train videos: {len(train_videos)}")
print(f"Val videos: {len(val_videos)}")
print(f"Test videos: {len(test_videos)}")

# Load the dataset
df = pd.read_csv(DATASET_PATH)
print(f"Dataset shape: {df.shape}")

# Display first few rows to understand the data
df.head()

# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

In [ ]:
# Process Labels (emotions)
# Convert string representation of emotion labels to lists
def process_labels(labels):
    if pd.isna(labels) or labels == 'None' or labels == '':
        return []
    try:
        # Try to evaluate as a literal Python expression (list, tuple, etc.)
        return eval(labels)
    except:
        # If it's a simple string without brackets
        if isinstance(labels, str):
            return [label.strip() for label in labels.split(',')]
        return []

# Apply to the Labels column
df['Processed_Labels'] = df['Labels'].apply(process_labels)

# List all possible emotions
all_emotions = ['Curiosity', 'Uncertainty', 'Excitement', 'Happiness', 'Surprise', 'Disgust', 'Fear', 'Frustration']
print(f"All possible emotions: {all_emotions}")

In [ ]:
# Convert Valence to numeric and handle missing values
df['Valence'] = pd.to_numeric(df['Valence'], errors='coerce')
# Fill missing valence with mean value
mean_valence = df['Valence'].mean()
df['Valence'] = df['Valence'].fillna(mean_valence)
print(f"Valence range: {df['Valence'].min()} to {df['Valence'].max()}")

# Create new column combining Transcript and Behavior
df['Combined_Text'] = df['Transcript'].fillna('') + ' ' + df['Behavior'].fillna('')
df['Combined_Text'] = df['Combined_Text'].str.strip()

# Handle cases where both transcript and behavior might be empty
df.loc[df['Combined_Text'] == '', 'Combined_Text'] = "No information available"


In [ ]:
# Split dataset into train, val, test based on the video files
def get_video_name(video_path):
    return os.path.basename(video_path)

df['VideoName'] = df['Video'].apply(get_video_name)

train_df = df[df['VideoName'].isin(train_videos)]
val_df = df[df['VideoName'].isin(val_videos)]
test_df = df[df['VideoName'].isin(test_videos)]

print(f"Train set: {len(train_df)} samples")
print(f"Val set: {len(val_df)} samples")
print(f"Test set: {len(test_df)} samples")

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=all_emotions)
mlb.fit([all_emotions])  # Ensure all classes are represented

# One-hot encode the labels
train_labels_encoded = mlb.transform(train_df['Processed_Labels'].tolist())
val_labels_encoded = mlb.transform(val_df['Processed_Labels'].tolist())
test_labels_encoded = mlb.transform(test_df['Processed_Labels'].tolist())

# Save the class mapping
with open(os.path.join(SAVE_DIR, 'emotion_labels.json'), 'w') as f:
    json.dump({idx: emotion for idx, emotion in enumerate(mlb.classes_)}, f)

In [ ]:
# Create a custom dataset class
class EmotionDataset(Dataset):
    def __init__(self, texts, emotion_labels, valence_values, tokenizer, max_length=512):
        self.texts = texts
        self.emotion_labels = emotion_labels
        self.valence_values = valence_values
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        emotion_label = self.emotion_labels[idx]
        valence = self.valence_values[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Squeeze to remove batch dimension
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'emotion_labels': torch.FloatTensor(emotion_label),
            'valence': torch.FloatTensor([valence])
        }

In [ ]:
from transformers import DistilBertModel
# Define the model
class EmotionValenceModel(nn.Module):
    def __init__(self, n_emotions, dropout_prob=0.3):
        super(EmotionValenceModel, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
        # Freeze some layers of DistilBERT to reduce computational load and prevent overfitting
        modules = [self.distilbert.embeddings, *self.distilbert.transformer.layer[:2]]
        for module in modules:
            for param in module.parameters():
                param.requires_grad = False
        
        hidden_size = self.distilbert.config.hidden_size
        
        # Common layers
        self.dropout = nn.Dropout(dropout_prob)
        self.pre_classifier = nn.Linear(hidden_size, 256)
        self.activation = nn.ReLU()
        
        # Emotion classification head
        self.emotion_classifier = nn.Linear(256, n_emotions)
        
        # Valence regression head
        self.valence_regressor = nn.Linear(256, 1)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0]  # Use [CLS] token representation
        
        x = self.dropout(hidden_state)
        x = self.pre_classifier(x)
        x = self.activation(x)
        x = self.dropout(x)
        
        # Multi-label emotion classification (using sigmoid for multi-label)
        emotion_output = self.emotion_classifier(x)
        
        # Valence regression (value between 1-7)
        valence_output = self.valence_regressor(x)
        
        return emotion_output, valence_output


In [ ]:
from transformers import DistilBertTokenizer
# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create datasets
train_dataset = EmotionDataset(
    train_df['Combined_Text'].tolist(),
    train_labels_encoded,
    train_df['Valence'].tolist(),
    tokenizer
)

val_dataset = EmotionDataset(
    val_df['Combined_Text'].tolist(),
    val_labels_encoded,
    val_df['Valence'].tolist(),
    tokenizer
)

test_dataset = EmotionDataset(
    test_df['Combined_Text'].tolist(),
    test_labels_encoded,
    test_df['Valence'].tolist(),
    tokenizer
)

In [ ]:
# Create data loaders
batch_size = 16  # Smaller batch size for M1 Mac

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model
n_emotions = len(all_emotions)
model = EmotionValenceModel(n_emotions)
model = model.to(device)

# Define loss functions
emotion_criterion = nn.BCEWithLogitsLoss()
valence_criterion = nn.MSELoss()

# Define optimizer with weight decay for regularization
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

# Learning rate scheduler
num_epochs = 50
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
# Training loop

def train_model(model, train_loader, val_loader, optimizer, scheduler, emotion_criterion, valence_criterion, num_epochs, device):
    # For tracking metrics
    history = {
        'train_emotion_loss': [],
        'train_valence_loss': [],
        'train_total_loss': [],
        'val_emotion_loss': [],
        'val_valence_loss': [],
        'val_total_loss': [],
        'val_micro_f1': [],
        'val_macro_f1': [],
        'val_valence_rmse': [],
        'val_accuracy': []  # Added accuracy tracking
    }
    
    # Alpha parameter for balancing losses (can be adjusted)
    alpha = 0.7  # Weight for emotion loss
    beta = 0.3   # Weight for valence loss
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        train_emotion_loss = 0
        train_valence_loss = 0
        train_total_loss = 0
        
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
        
        for batch_idx, batch in progress_bar:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            emotion_labels = batch['emotion_labels'].to(device)
            valence = batch['valence'].to(device)
            
            # Forward pass
            emotion_logits, valence_pred = model(input_ids, attention_mask)
            
            # Calculate losses
            e_loss = emotion_criterion(emotion_logits, emotion_labels)
            v_loss = valence_criterion(valence_pred, valence)
            
            # Weighted combined loss
            loss = alpha * e_loss + beta * v_loss
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            
            # Update metrics
            train_emotion_loss += e_loss.item()
            train_valence_loss += v_loss.item()
            train_total_loss += loss.item()
            
            # Update progress bar
            progress_bar.set_description(f"Training - Loss: {loss.item():.4f}")
        
        # Calculate average training losses
        avg_train_emotion_loss = train_emotion_loss / len(train_loader)
        avg_train_valence_loss = train_valence_loss / len(train_loader)
        avg_train_total_loss = train_total_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_emotion_loss = 0
        val_valence_loss = 0
        val_total_loss = 0
        
        all_emotion_preds = []
        all_emotion_labels = []
        all_valence_preds = []
        all_valence_labels = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                emotion_labels = batch['emotion_labels'].to(device)
                valence = batch['valence'].to(device)
                
                emotion_logits, valence_pred = model(input_ids, attention_mask)
                
                e_loss = emotion_criterion(emotion_logits, emotion_labels)
                v_loss = valence_criterion(valence_pred, valence)
                loss = alpha * e_loss + beta * v_loss
                
                val_emotion_loss += e_loss.item()
                val_valence_loss += v_loss.item()
                val_total_loss += loss.item()
                
                # Convert logits to predictions
                emotion_preds = torch.sigmoid(emotion_logits).cpu().numpy() > 0.5  # Threshold for multi-label classification
                # Convert lists to numpy arrays for F1 calculation
                
                all_emotion_preds.extend(emotion_preds.astype(int).tolist())
                all_emotion_labels.extend(emotion_labels.cpu().numpy().astype(int).tolist())
                all_valence_preds.extend(valence_pred.cpu().numpy().flatten().tolist())
                all_valence_labels.extend(valence.cpu().numpy().flatten().tolist())
        
        # Calculate validation metrics
        avg_val_emotion_loss = val_emotion_loss / len(val_loader)
        avg_val_valence_loss = val_valence_loss / len(val_loader)
        avg_val_total_loss = val_total_loss / len(val_loader)
        
        # F1 scores for emotion classification
        #micro_f1 = f1_score(all_emotion_labels, all_emotion_preds, average='micro')
        # macro_f1 = f1_score(all_emotion_labels, all_emotion_preds, average='macro')
        # Convert to numpy arrays
        emotion_preds_np = np.array(all_emotion_preds)
        emotion_labels_np = np.array(all_emotion_labels)

        # F1 scores for multi-label emotion classification
        micro_f1 = f1_score(emotion_labels_np, emotion_preds_np, average='micro', zero_division=0)
        macro_f1 = f1_score(emotion_labels_np, emotion_preds_np, average='macro', zero_division=0)
        
        # RMSE for valence prediction
        valence_rmse = np.sqrt(mean_squared_error(all_valence_labels, all_valence_preds))
        
        # Calculate accuracy for multi-label classification
        accuracy = np.mean(np.array(emotion_preds_np == emotion_labels_np, dtype=float))
        
        # Update history
        history['train_emotion_loss'].append(avg_train_emotion_loss)
        history['train_valence_loss'].append(avg_train_valence_loss)
        history['train_total_loss'].append(avg_train_total_loss)
        history['val_emotion_loss'].append(avg_val_emotion_loss)
        history['val_valence_loss'].append(avg_val_valence_loss)
        history['val_total_loss'].append(avg_val_total_loss)
        history['val_micro_f1'].append(micro_f1)
        history['val_macro_f1'].append(macro_f1)
        history['val_valence_rmse'].append(valence_rmse)
        history['val_accuracy'].append(accuracy)  # Added accuracy to history
        
        # Print metrics
        print(f"Train - Emotion Loss: {avg_train_emotion_loss:.4f}, Valence Loss: {avg_train_valence_loss:.4f}, Total Loss: {avg_train_total_loss:.4f}")
        print(f"Val - Emotion Loss: {avg_val_emotion_loss:.4f}, Valence Loss: {avg_val_valence_loss:.4f}, Total Loss: {avg_val_total_loss:.4f}")
        print(f"Val - Micro F1: {micro_f1:.4f}, Macro F1: {macro_f1:.4f}, Valence RMSE: {valence_rmse:.4f}, Accuracy: {accuracy:.4f}")  # Added accuracy to print statement
        
        # Save best model
        if avg_val_total_loss < best_val_loss:
            best_val_loss = avg_val_total_loss
            torch.save(model.state_dict(), os.path.join(SAVE_DIR, 'dbert_best_model.pt'))
            print("Saved best model!")
    
    # Save final model
    torch.save(model.state_dict(), os.path.join(SAVE_DIR, 'dbert_final_model.pt'))
    
    # Save training history
    with open(os.path.join(SAVE_DIR, 'dbert_training_history.json'), 'w') as f:
        json.dump(history, f)
    
    return history


In [ ]:
# Train the model
history = train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    emotion_criterion,
    valence_criterion,
    num_epochs,
    device
)

In [ ]:

  # Plot the training history
def plot_training_history(history):
    # Create subplots - changed to 2x3 grid to add accuracy plot
    fig, axs = plt.subplots(2, 3, figsize=(18, 10))
    
    # Plot total loss
    axs[0, 0].plot(history['train_total_loss'], label='Train Loss')
    axs[0, 0].plot(history['val_total_loss'], label='Val Loss')
    axs[0, 0].set_title('Total Loss')
    axs[0, 0].set_xlabel('Epoch')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].legend()
    
    # Plot emotion loss
    axs[0, 1].plot(history['train_emotion_loss'], label='Train Emotion Loss')
    axs[0, 1].plot(history['val_emotion_loss'], label='Val Emotion Loss')
    axs[0, 1].set_title('Emotion Loss')
    axs[0, 1].set_xlabel('Epoch')
    axs[0, 1].set_ylabel('Loss')
    axs[0, 1].legend()
    
    # Plot valence loss and RMSE
    axs[0, 2].plot(history['train_valence_loss'], label='Train Valence Loss')
    axs[0, 2].plot(history['val_valence_loss'], label='Val Valence Loss')
    axs[0, 2].plot(history['val_valence_rmse'], label='Val Valence RMSE')
    axs[0, 2].set_title('Valence Loss & RMSE')
    axs[0, 2].set_xlabel('Epoch')
    axs[0, 2].set_ylabel('Loss / RMSE')
    axs[0, 2].legend()
    
    # Plot F1 scores
    axs[1, 0].plot(history['val_micro_f1'], label='Micro F1')
    axs[1, 0].plot(history['val_macro_f1'], label='Macro F1')
    axs[1, 0].set_title('F1 Scores')
    axs[1, 0].set_xlabel('Epoch')
    axs[1, 0].set_ylabel('F1 Score')
    axs[1, 0].legend()
    
    # Plot accuracy
    axs[1, 1].plot(history['val_accuracy'], label='Accuracy', color='purple')
    axs[1, 1].set_title('Model Accuracy')
    axs[1, 1].set_xlabel('Epoch')
    axs[1, 1].set_ylabel('Accuracy')
    axs[1, 1].set_ylim([0, 1])  # Accuracy is between 0 and 1
    axs[1, 1].legend()
    
    # Hide the unused subplot
    axs[1, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, 'dbert_training_history.png'))
    plt.show()

# Plot the training history
plot_training_history(history)

In [ ]:
# Evaluate the model on the test set
def evaluate_model(model, test_loader, emotion_criterion, valence_criterion, device):
    model.eval()
    test_emotion_loss = 0
    test_valence_loss = 0
    all_emotion_preds = []
    all_emotion_labels = []
    all_valence_preds = []
    all_valence_labels = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            emotion_labels = batch['emotion_labels'].to(device)
            valence = batch['valence'].to(device)
            
            emotion_logits, valence_pred = model(input_ids, attention_mask)
            
            e_loss = emotion_criterion(emotion_logits, emotion_labels)
            v_loss = valence_criterion(valence_pred, valence)
            
            test_emotion_loss += e_loss.item()
            test_valence_loss += v_loss.item()
            
            # Convert logits to predictions
            emotion_preds = torch.sigmoid(emotion_logits).cpu().numpy() > 0.5 
            
            all_emotion_preds.extend(emotion_preds.astype(int).tolist())
            all_emotion_labels.extend(emotion_labels.cpu().numpy().astype(int).tolist())
            all_valence_preds.extend(valence_pred.cpu().numpy().flatten().tolist())
            all_valence_labels.extend(valence.cpu().numpy().flatten().tolist())
            
    # Calculate test metrics
    avg_test_emotion_loss = test_emotion_loss / len(test_loader)
    avg_test_valence_loss = test_valence_loss / len(test_loader)
    
    # F1 scores for emotion classification
    # Convert to numpy arrays
    emotion_preds_np = np.array(all_emotion_preds)
    emotion_labels_np = np.array(all_emotion_labels)
    
    # F1 scores for multi-label emotion classification
    micro_f1 = f1_score(emotion_labels_np, emotion_preds_np, average='micro', zero_division=0)
    macro_f1 = f1_score(emotion_labels_np, emotion_preds_np, average='macro', zero_division=0)
    
    # F1 scores for each class
    class_f1 = f1_score(all_emotion_labels, all_emotion_preds, average=None)
    class_report = {emotion: score for emotion, score in zip(all_emotions, class_f1)}
    
    # RMSE for valence prediction
    valence_rmse = np.sqrt(mean_squared_error(all_valence_labels, all_valence_preds))
    
    # Calculate accuracy for multi-label classification
    accuracy = np.mean(np.array(emotion_preds_np == emotion_labels_np, dtype=float))
    
    # Create a confusion matrix for multi-label is complex
    # Instead, analyze per-class precision, recall
    results = {
        'test_emotion_loss': avg_test_emotion_loss,
        'test_valence_loss': avg_test_valence_loss,
        'test_micro_f1': micro_f1,
        'test_macro_f1': macro_f1,
        'test_valence_rmse': valence_rmse,
        'test_accuracy': accuracy,  # Added accuracy
        'class_f1': class_report
    }
    
    # Print the results
    print(f"Test Results:")
    print(f"Emotion Loss: {avg_test_emotion_loss:.4f}")
    print(f"Valence Loss: {avg_test_valence_loss:.4f}")
    print(f"Micro F1: {micro_f1:.4f}")
    print(f"Macro F1: {macro_f1:.4f}")
    print(f"Valence RMSE: {valence_rmse:.4f}")
    print(f"Accuracy: {accuracy:.4f}")  # Added accuracy print statement
    
    # Save results
    with open(os.path.join(SAVE_DIR, 'dbert_test_results.json'), 'w') as f:
        json.dump(results, f)
    
    return results

In [ ]:
# Load the best model for evaluation
model.load_state_dict(torch.load(os.path.join(SAVE_DIR, 'dbert_best_model.pt')))

# Evaluate the model
test_results = evaluate_model(model, test_loader, emotion_criterion, valence_criterion, device)

print("\n")
# Print test results
'''A
print("\nTest Results:")
print(f"Emotion Loss: {test_results['test_emotion_loss']:.4f}")
print(f"Valence Loss: {test_results['test_valence_loss']:.4f}")
print(f"Micro F1: {test_results['test_micro_f1']:.4f}")
print(f"Macro F1: {test_results['test_macro_f1']:.4f}")
print(f"Valence RMSE: {test_results['test_valence_rmse']:.4f}")
print(f"Accuracy: {test_results['test_accuracy']:.4f}")
'''
print("\nClass-wise F1 Scores:")
for emotion, f1 in test_results['class_f1'].items():
    print(f"{emotion}: {f1:.4f}")

In [ ]:
# Function to create and save radar chart
def create_emotion_radar_chart(class_f1, emotions, save_dir):
    # Number of variables
    N = len(emotions)
    
    # Get F1 scores in same order as emotions
    f1_scores = [class_f1[emotion] for emotion in emotions]
    
    # What will be the angle of each axis in the plot
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # F1 scores for plotting (add first element at end to close the loop)
    values = f1_scores.copy()
    values += values[:1]  
    
    # Initialize the figure
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, polar=True)
    
    # Draw one axis per variable and add labels
    plt.xticks(angles[:-1], emotions, fontsize=12)
    
    # Draw the y-axis labels
    ax.set_rlabel_position(0)
    plt.yticks([0.2, 0.4, 0.6, 0.8, 1.0], ["0.2", "0.4", "0.6", "0.8", "1.0"], 
               fontsize=10, color="grey")
    plt.ylim(0, 1)
    
    # Plot data
    ax.plot(angles, values, linewidth=2, linestyle='solid', color='#FF7F0E')
    
    # Fill area
    ax.fill(angles, values, alpha=0.25, color='#FF7F0E')
    
    # Add title
    plt.title("Emotion-wise F1 Scores", size=20, y=1.05)
    
    # Add grid lines
    ax.grid(True)
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'dbert_emotion_F1s.png'))
    plt.show()
create_emotion_radar_chart(test_results["class_f1"], all_emotions, SAVE_DIR)

In [ ]:
# Function to make predictions for a new input
def predict_emotions_and_valence(text, model, tokenizer, mlb, device):
    model.eval()
    
    # Tokenize the input text
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        emotion_logits, valence_pred = model(input_ids, attention_mask)
        
        # Convert logits to predictions
        emotion_probs = torch.sigmoid(emotion_logits).cpu().numpy()
        emotion_preds = emotion_probs > 0.5
        
        # Get the predicted emotions
        predicted_emotions = mlb.inverse_transform(emotion_preds)[0]
        
        # Get the predicted valence
        predicted_valence = valence_pred.cpu().numpy().item()
        
        # Get the probabilities for each emotion
        emotion_probabilities = {emotion: prob for emotion, prob in zip(mlb.classes_, emotion_probs[0])}
    
    return predicted_emotions, predicted_valence, emotion_probabilities


In [ ]:
# Example prediction function
test_text = "I am really excited about this new movie. The trailer looks amazing and I can't wait to see it!"
predicted_emotions, predicted_valence, emotion_probs = predict_emotions_and_valence(
    test_text, model, tokenizer, mlb, device
)

print("\nExample Prediction:")
print(f"Input text: {test_text}")
print(f"Predicted emotions: {predicted_emotions}")
print(f"Predicted valence: {predicted_valence:.2f}")
print("Emotion probabilities:")
for emotion, prob in emotion_probs.items():
    print(f"{emotion}: {prob:.4f}")

In [ ]:

# Save the complete model, tokenizer, and configuration
model_save_path = os.path.join(SAVE_DIR, 'dbert')
os.makedirs(model_save_path, exist_ok=True)

# Save model configuration
model_config = {
    'model_name': 'distilbert-base-uncased',
    'n_emotions': len(all_emotions),
    'emotions': all_emotions,
    'valence_range': [df['Valence'].min(), df['Valence'].max()],
    'max_length': 512,
}

with open(os.path.join(model_save_path, 'config.json'), 'w') as f:
    json.dump(model_config, f)

# Save the model state
torch.save(model.state_dict(), os.path.join(model_save_path, 'model_state.pt'))

# Save tokenizer
tokenizer.save_pretrained(model_save_path)

print(f"\nModel, tokenizer, and configuration saved to {model_save_path}")


In [ ]:
# Final messages
print("\nTraining completed successfully!")
print(f"Best model saved to: {os.path.join(SAVE_DIR, 'dbert_best_model.pt')}")
print(f"Final model saved to: {os.path.join(SAVE_DIR, 'dbert_final_model.pt')}")
print(f"Complete model package saved to: {model_save_path}")
print(f"Training history plot saved to: {os.path.join(SAVE_DIR, 'dbert_training_history.png')}")
print(f"Training history saved to: {os.path.join(SAVE_DIR, 'dbert_training_history.json')}")
print(f"Test results saved to: {os.path.join(SAVE_DIR, 'dbert_test_results.json')}")